# MeowLab

> cat可能是你最常用的命令行工具之一，但是可能很少有人会关注cat的性能问题。
> 
> 但实际上，cat可能比你想的更加高效。在这个lab中，我们会一起来探索cat如此高效的秘密。

## 什么是cat

对于大部分同学来说，这个问题有些太trivial了。但是以防万一，还是在这里介绍一下，cat是一个GNU coreutils中的一个程序，它的作用是连接(con**cat**enate)一系列文件，并将其输出到标准输出流(stdout)中。

> 如果你愿意去找，你会发现这个lab中所有的代码都可以在GNU coreutils中找到，或者你用某个大模型，它可能也能写得不错。
> 但是，除了写代码以外，这个lab中也会包含一些分析和讨论。因此即使你选择直接去抄GNU coreutils中的代码，也不要直接不加思考的复制粘贴。

## 环境要求

* 操作系统：Linux（任何包含GNU coreutils的发行版）
* 编译器：本lab允许使用C/C++或者Rust，选择你喜欢的任何编译器即可。
* Python3.x：本lab一些地方需要你画图，推荐你使用matplotlib。另外，我们使用一个简单的脚本来生成测试文件。

## 在开始之前

这个项目的所有代码需要你自己动手来写，我们只提供了一个用于生成测试文件的脚本。

为了展示比较明显的测试效果，我们会生成一个2GB的测试文件，请确保你的磁盘拥有足够的空间。你可以运行下面这个单元格来查看你的电脑是否有足够的空间。这里我们使用了`df`命令，这个命令的作用是查看某个文件系统所在的磁盘的使用量。

In [ ]:
%%bash
df -h /

avail=$(df -h / | awk 'NR==2 {print $4}' | grep -o '[0-9.]*')
unit=$(df -h / | awk 'NR==2 {print $4}' | grep -o '[a-zA-Z]*')
if [[ "$unit" == "M" || "$unit" == "K" ]]; then
    echo "Disk space is low: $avail$unit"
elif [[ "$unit" == "T" ]]; then
    echo "Disk space is sufficient: $avail$unit"
elif [[ "$unit" == "G" ]]; then
    if (( $(echo "$avail < 10" | bc -l) )); then
        echo "Disk space is low: $avail$unit"
    else
        echo "Disk space is sufficient: $avail$unit"
    fi
else
    echo "Unknown unit: $unit"
fi


你可以使用我们提供的python脚本来生成测试文件，运行下面的单元格。测试文件的生成可能会花费一定的时间。

In [ ]:
import random

MB = 1024 * 1024

# A static seed for reproducibility
random.seed(42)

with open("test.txt", "wb") as f:
    for _ in range(2048):
        f.write(random.randbytes(1 * MB)) # 1MB of random data

当前文件夹下面会出现一个大小为2GB的文件`test.txt`。你可以运行下面的单元格来验证。

In [ ]:
%%bash
ls -lh test.txt

最后，我们的lab会需要使用`hyperfine`来测量程序的运行时间。我们推荐你使用`cargo`进行安装。你可以从[Rust官网](https://www.rust-lang.org/zh-CN/learn/get-started)找到安装脚本。如果你的电脑已经安装好了`cargo`，你可以运行下面的单元格来安装`hyperfine`。

In [ ]:
%%bash
cargo install hyperfine

有了以上的准备工作，我们终于可以开始我们的探索之旅了。

## 任务0: 测量cat的效率

你要做的第一个任务十分简单。学习一下如何使用`hyperfine`，并且使用它来测试GNU coreutils中的cat输出我们生成的测试文件`test.txt`的时间。运行下面的单元格来运行测试。

In [ ]:
%%bash
hyperfine --warmup 3 'cat test.txt'

## 任务1: 写一个最朴素的cat

我们现在使用`hyperfine`测量了GNU coreutils中的`cat`的运行效率，但是对于这个运行时间，我们可能并没有什么概念。现在我们将会写一个最朴素的`cat`程序来作为baseline。这个程序需要满足一下要求：
1. 把你的源代码命名为`mycat1.c`或者`mycat1.rs`，并且可执行的二进制文件命名为`mycat1`。
2. 你的程序接受且只接受一个命令行参数，这个参数是你要读取的文件。
3. 你的程序直接使用`read`和`write`系统调用，每次读取并输出一个字符，不使用任何缓冲区。
4. 使用`hpyerfine`测量`mycat1`和`cat`运行时间（`mycat1`的运行时间可能会非常长）
5. 虽然这是一个很简单的程序，但是请仍然保持系统编程的规范。请在你的代码中进行进行所有必要的错误处理。

这个任务不需要在文档中包含任何内容。

In [ ]:
%%bash
hyperfine --warmup 3 --runs 1 './target/mycat1 test.txt'

## 任务2: 带有缓冲区的cat

如果你正确完成了上面的任务，你会发现，`mycat1`的性能和系统中的`cat`的性能相差甚远。但是经过了ics课程的学习，你肯定已经想到了至少一个解决方案——使用缓冲区。缓冲区可以有效减少反复进行系统调用的性能消耗。但是，我们要面临的第二个问题是：缓冲区的大小应该如何设置呢？我们接下来将一步一步地探索这个问题。

我们之前肯定也已经讲过不少涉及IO缓冲区的程序了，它们的缓冲区大小都被设置为多少呢？我相信1024, 4096, 8192一定是比较常见的几个数字。经过ics的学习，我相信你们肯定能说出原因了。那么，这个任务，我们将根据这个原理优化我们的`mycat1`。你至少需要完成下面要求：
1. 复制你上一个任务的源代码在上面做修改，并把源代码命名为`mycat2.c`/`mycat2.rs`，可执行二进制文件命名为`mycat2`。
2. 写一个`io_blocksize`函数用于确定你的缓冲区的大小，在这个任务中，你可以将缓冲区设置成你当前系统中一个内存页的大小。（注意：你不能假设所有系统中内存页的大小都是4K，请你使用一个系统调用或者标准库中的函数来获取内存页的大小，而不要使用一个固定值。不过允许你使用一个固定值，如果获取内存页发生错误，但是这不应该是一个常见的情况）。
3. 使用标准库提供的函数动态分配用于缓冲区的内存。
4. 使用`hpyerfine`测量`mycat2`的运行时间
5. 请保持系统编程的基本规范。

这个任务不需要在文档中包含任何内容。

In [ ]:
%%bash
hyperfine --warmup 3 './target/mycat2 test.txt'

## 任务3: 缓冲区对齐的cat

如果你正确完成了上面这个任务，你会发现，添加了缓冲区的`mycat2`性能提升十分显著。但是我们还可以进一步优化。实际上只是添加了缓冲区并且设置缓冲区的大小为内存页的整数倍并不是没有什么太多的意义，这样的设置只是为了这个一个任务做铺垫的。在这个任务中，我们将尝试将我们的缓冲区对齐到系统的内存页。至于为什么要这么做，请大家在自己的文档中分析一下。你至少需要完成以下要求：
1. 复制你上一个任务的源代码在上面做修改，并把源代码命名为`mycat3.c`/`mycat3.rs`，可执行二进制文件命名为`mycat3`。
2. 写两个函数`char* align_alloc(size_t size)`和`void align_free(void* ptr)`，它们的作用分别是分配一段内存，长度不小于`size`并且返回一个对齐到内存页起始的指针`ptr`，以及给出一个先前从`align_alloc`返回的指针并释放之前分配的内存。
3. 利用这两个函数修改你的代码，缓冲区的大小仍然设置成一个内存页的大小。
4. 使用`hpyerfine`测量`mycat3`的运行时间
5. 请保持系统编程的基本规范。

这个任务，你需要在文档中回答以下问题：
1. 为什么将缓冲区对齐到系统的内存可能提高性能？你的实验结果支持这个猜想吗？为什么？
2. 为什么我们直接使用`malloc`函数分配的内存不能对齐到内存页，即使我们分配的内存大小已经是内存页大小的整数倍了。
3. 你是怎么在不知道原始的malloc返回的指针的情况下正确释放内存的？

1、减少TLB未命中：对齐内存页边界减少转换后备缓冲器(TLB)未命中次数
    优化DMA操作：直接内存访问(DMA)需要物理连续内存，对齐缓冲区满足此要求
    避免缓存行分裂：防止单个缓存行跨越两个内存页
    减少页错误：完整页面访问减少次要页错误发生
    实验结果支持猜想，mycat2耗时347ms+-79.1ms，mycat3耗时279ms+-4ms
    原因：对齐优化主要受益于现代CPU的内存管理单元(MMU)
        在虚拟化环境中效果更显著，因为减少了嵌套分页开销
        当系统内存压力大时，对齐内存更易被高效管理


2、内存分配元数据：
malloc在分配的内存块前存储管理信息（大小、状态等）
这些元数据通常占用16-32字节，导致返回的指针不位于页边界
对齐保证差异：
malloc只保证基本对齐（通常是8或16字节对齐）
内存池设计：
分配器使用内存池策略，从大块内存中分割
即使请求大块内存，起始地址可能不在页边界


3、使用指针存储技术：
在对齐指针前存储原始指针
额外空间包含：对齐间隙 + 指针存储空间
释放时从已知偏移量获取原始指针
![result3](./mycat3.png)

In [ ]:
%%bash
hyperfine --warmup 3 './target/mycat3 test.txt'

## 任务4: 设置缓冲区大小为文件系统块大小的整数倍的cat

由于`cat`是涉及文件操作的，所以我们自然不能离开磁盘操作。我们在课内已经学到过，磁盘操作的基本单位是块。并且因为我们操作磁盘是经过了操作系统的一层抽象的，操作系统的文件系统也定义了一个操作文件的基本单位块，这个块的大小和磁盘的块的大小相关，但不总是相同。因此我们操作文件的时候实际接触到的块大小是文件系统的块大小。如果我们每次读取和写入文件的时候都按照文件系统的块大小来进行，也能提升性能。在这个任务中，你至少需要完成以下要求：
1. 复制你上一个任务的源代码在上面做修改，并把源代码命名为`mycat4.c`/`mycat4.rs`，可执行二进制文件命名为`mycat4`。
2. 修改你的函数`io_blocksize`，让你的缓冲区大小既考虑到内存页大小也考虑到文件系统的块大小。
3. 使用`hyperfine`测量`mycat4`的运行时间。
4. 保持系统编程的基本规范。

> 在完成这项任务的时候你需要注意以下几点：
> 1. 文件系统中的每个文件，块大小不总是相同的。
> 2. 有的文件系统可能会给出虚假的块大小，这种虚假的文件块大小可能根本不是2的整数次幂。

这个任务，你需要在文档中回答以下问题：
1. 为什么在设置缓冲区大小的时候需要考虑到文件系统块的大小的问题？
2. 对于上面提到的两个注意事项你是怎么解决的？

1：

（1）I/O效率优化：文件系统以块为单位管理数据，匹配块大小确保每次读取完整块，避免"部分块读取"导致的额外I/O操作


（2）减少内部碎片：缓冲区大小非块大小整数倍时，导致缓冲区尾部空间浪费，增加无效数据传输


（3）硬件层优化：磁盘控制器以块为单位操作，SSD擦除块大小影响写放大，RAID条带大小需要匹配
（4）预读机制协同：文件系统预读通常基于块大小，匹配大小使预读更有效


2：
（1）使用fstat获取具体文件的块大小，每次打开文件时重新获取，不依赖全局文件系统信息


（2）验证块大小为2的幂，设置合理范围限制（1KB-1MB），回退到默认页大小
![result4](./mycat4.png)

In [ ]:
%%bash
hyperfine --warmup 3 './target/mycat4 test.txt'

## 任务5: 考虑系统调用开销情况下的cat

如果你正确完成了上面的任务，那么现在你的`cat`已经可以综合考虑内存页大小，内存页对齐和文件系统块大小的因素来设置缓冲区大小了。但是我们会发现，我们自己的`cat`性能仍然不如我们系统中的`cat`。并且如果你尝试过再进一步增大缓冲区的大小，你的`cat`性能还能更高。这是因为我们目前设置的缓冲区大小还不足以掩盖系统调用带来的开销。那么，我们的缓冲区究竟应该设置到什么大小才够呢？其实这个问题，我们仅仅使用理论分析是无法给出答案的，因为答案受到机器的硬件条件，操作系统的涉及等多重不确定因素的影响。但是，我们可以使用实验来解答这个问题。最后，我们还需要做出假设，不考虑上一个任务的注意事项1，也就是我们假设我们文件系统的大部分文件的块大小都一致（你可以使用我们的测试文件的块大小）。因此，设我们在之前的任务中设置的缓冲区大小是buf_size，我们需要通过实验找到一个倍数A，满足以下条件：
1. 当缓冲区大小小于A * buf_size的时候，文件的读写速度显著减小
2. 当缓冲区大小大于A * buf_size的时候，文件的读写速度并不显著提升
最终，我们就可以直接设置我们的`cat`中的缓冲区大小设置成buf_size的固定倍率。在这个任务中，你只少需要完成以下要求：
1. 编写一个实验脚本，尽量排除其他因素的影响，测量只在系统调用开销的影响下，你的系统最合适的缓冲区大小。并且给出这个大小下你的系统的读写速率。
2. 复制你上一个任务的源代码在上面做修改，并把源代码命名为`mycat5.c`/`mycat5.rs`，可执行二进制文件命名为`mycat5`。
3. 利用上面的实验结果，修改你的函数`io_blocksize`。
4. 使用`hyperfine`测量`mycat5`的运行时间。
5. 保持系统编程的基本规范。

> 提示：
> 1. `dd`命令可以用于复制一个文件(也就是读取并写入)，你可以使用命令行参数设置它的缓冲区大小，并且程序终止的时候可以报告平均文件读写速度。
> 2. Linux系统中存在`/dev`文件系统，这个目录下有很多特殊文件，其中有一些对你来说可能有用。`/dev/null`，你向这个文件写入的内容并不真正写入磁盘，并且不会对你的系统造成任何影响。`/dev/zero`，如果你尝试读取这个文件你会得到源源不断的`\0x0`，这个文件也并不真正的从磁盘中读取。

这个任务，你需要在文档中包括以下内容：
1. 解释一下你的实验脚本是怎么设计的。你应该尝试了多种倍率，请将它们的读写速率画成图表包含在文档中。

1、测试方法：使用dd命令复制2GB测试文件
改变bs(block size)参数从4KB到4MB
每次测试运行3次取平均值
测量实际执行时间，计算吞吐量(MB/s)


2、参数范围：buffer_sizes = [4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]
缓冲区大小 vs 读取速度
====================================================================================================
4       KB | ███████████████████████████ 4032.14 MB/s
8       KB | ████████████████████████████████████████████████████████████ 9035.65 MB/s
16      KB | █████████████████████████████████████████████████████████████████████ 10297.57 MB/s
32      KB | █████████████████████████████████████████████████████████████████████████ 10942.13 MB/s
64      KB | ████████████████████████████████████████████████████████████████████████████ 11391.58 MB/s
128     KB | ██████████████████████████████████████████████████████████████████████████████ 11626.94 MB/s
256     KB | ████████████████████████████████████████████████████████████████████████████ 11406.32 MB/s
512     KB | ████████████████████████████████████████████████████████████████████████████████ 11873.93 MB/s
1024    KB | ██████████████████████████████████████████████████████████████████████████████ 11645.48 MB/s
2048    KB | ████████████████████████████████████████████████████████████████████████ 10722.63 MB/s
4096    KB | ██████████████████████████████████████████████████████████████████████████ 11067.00 MB/s
====================================================================================================

最佳缓冲区大小: 512KB (速度: 11873.93 MB/s)

In [ ]:
import subprocess
import csv
import os
import time
import sys

def run_dd_test(buffer_size_kb, runs=3):
    """运行 dd 测试并返回平均速度 (MB/s)"""
    total_time = 0.0
    success_runs = 0
    
    for i in range(runs):
        try:
            # 准备命令
            cmd = f"dd if=test.txt of=/dev/null bs={buffer_size_kb}k status=none"
            
            # 直接使用 Python 计时
            start_time = time.time()
            subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            end_time = time.time()
            
            # 计算执行时间
            elapsed = end_time - start_time
            total_time += elapsed
            success_runs += 1
            
            print(f"运行 {i+1}/{runs}: {elapsed:.2f}s", end=' ', flush=True)
        
        except subprocess.CalledProcessError as e:
            print(f"dd 命令执行失败: {e}")
        except Exception as e:
            print(f"运行测试时出错: {e}")
    
    if success_runs == 0:
        print(f"错误：所有 {runs} 次运行均失败，缓冲区大小: {buffer_size_kb}KB")
        return 0.0
    
    avg_time = total_time / success_runs
    
    # 确保时间不为零
    if avg_time < 0.001:  # 小于1毫秒
        print(f"警告：测试时间过短 ({avg_time:.6f}s)，使用0.001s作为下限")
        avg_time = 0.001
    
    # 2GB = 2048 MB
    speed = 2048 / avg_time  # MB/s
    return speed

def generate_text_chart(data, title, width=80):
    """生成纯文本柱状图"""
    if not data:
        print("无数据可用于生成图表")
        return
    
    # 过滤无效数据点
    valid_data = [(str(label), val) for label, val in data if val > 0]
    
    if not valid_data:
        print("无有效数据可用于生成图表")
        return
    
    max_value = max(val for _, val in valid_data)
    scale = width / max_value
    
    print(f"\n{title}")
    print("=" * (width + 20))
    
    for label, value in valid_data:
        bar_length = int(value * scale)
        bar = '█' * bar_length
        print(f"{label:8}KB | {bar} {value:.2f} MB/s")
    
    print("=" * (width + 20))

def save_to_csv(data, filename):
    """保存数据到 CSV 文件"""
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Buffer Size (KB)", "Speed (MB/s)"])
        writer.writerows(data)
    print(f"数据已保存到 {filename}")

def main():
    print("开始缓冲区大小性能测试...")
    
    # 1. 检查测试文件是否存在
    test_file = "test.txt"
    if not os.path.exists(test_file):
        print(f"错误：测试文件 {test_file} 不存在！")
        print("请先运行生成测试文件的代码单元格")
        return
    
    # 2. 检查文件大小
    file_size = os.path.getsize(test_file)
    file_size_gb = file_size / (1024 ** 3)
    if file_size_gb < 1.5 or file_size_gb > 2.5:
        print(f"警告：测试文件大小应为2GB，实际为 {file_size_gb:.2f}GB")
    
    # 3. 测试不同缓冲区大小的性能
    buffer_sizes = [4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]
    results = []
    
    print("\n测试不同缓冲区大小...")
    for size_kb in buffer_sizes:
        print(f"\n测试缓冲区: {size_kb}KB...", flush=True)
        speed = run_dd_test(size_kb)
        results.append((size_kb, speed))
        print(f"平均速度: {speed:.2f} MB/s")
    
    # 4. 处理结果
    if any(speed == 0 for _, speed in results):
        print("\n警告：某些测试点返回零速度，数据可能不准确")
    
    # 5. 保存结果到 CSV
    save_to_csv(results, "buffer_performance.csv")
    
    # 6. 显示文本图表
    generate_text_chart(results, "缓冲区大小 vs 读取速度")
    
    # 7. 找出最佳缓冲区大小
    try:
        best_size, best_speed = max(results, key=lambda x: x[1])
        print(f"\n最佳缓冲区大小: {best_size}KB (速度: {best_speed:.2f} MB/s)")
    except:
        print("\n无法确定最佳缓冲区大小")
    
    print("\n所有测试完成!")

if __name__ == "__main__":
    main()

In [ ]:
%%bash
hyperfine --warmup 3 './target/mycat5 test.txt'

## 任务6: 使用了系统调用`fdadvice`的cat

虽然几乎我们的这个lab都在讨论设置缓冲区大小的问题，但是实际上我们的系统中的`cat`还在其他很多方面做了不少优化。这些优化在多数时候和缓冲区相比都不起决定性作用，但是我们也可以从中学习到不少有趣的技巧。这里我们就只尝试其中一个，使用系统调用`fadvise`。这个系统调用可以提示文件系统我们将会以什么样的模式来读写文件，这样操作系统可以设置合适的readahead窗口为文件的读写做准备。在这个任务中，你需要完成以下要求：
1. 复制你上一个任务的源代码在上面做修改，并把源代码命名为`mycat6.c`/`mycat6.rs`，可执行二进制文件命名为`mycat6`。
2. 在你的代码中使用`fadvise`进行优化。
3. 使用`hyperfine`测量`mycat6`的运行时间。
4. 保持系统编程的基本规范。

这个任务，你需要在文档中回答以下问题：
1. 你是如何设置`fadvise`的参数的？
2. 对于顺序读写的情况，文件系统可以如何调整readahead？对于随机读写的情况呢？

1、posix_fadvise(fd, 0, 0, POSIX_FADV_SEQUENTIAL);


fd：打开的文件描述符


0：偏移量起始位置（文件开头）


0：长度（0表示整个文件）


POSIX_FADV_SEQUENTIAL：顺序访问模式标志

参数选择原因：
适合cat的线性读取模式
最大化预读效果
优化页面缓存回收策略

2、顺序读写情况：
增大预读窗口：
初始值：128KB
检测顺序模式后：逐步增加到1-2MB


主动预读：
在应用请求前预取后续数据
使用空闲I/O带宽后台读取


缓存优化：
标记页面为顺序访问
优先回收已读页面


I/O调度：
合并相邻请求
按物理位置排序请求


随机读写情况：
减少预读：
最小化预读窗口（通常64KB）
严格按请求读取


缓存策略：
保留最近访问页面更长时间
使用更复杂的替换算法


I/O调度：
基于位置排序请求
优先级调度

In [ ]:
%%bash
hyperfine --warmup 3 './target/mycat6 test.txt'

## 任务7: 总结

经过了上面的所有任务，我们已经成功探索我们系统中最常用的工具`cat`所使用的各种优化。我相信你对涉及系统IO的编程也有了更深刻的理解。现在请你整理汇总上面每个任务你所写的`mycatx`以及系统的`cat`的测量数据，使用一个柱状图来展示。并且请你分析一下你得到的结果：它们符合你的预期吗？为什么？这个结果给你带来了什么启示？

这个任务，你需要在文档中包含以下内容：
1. 你的全部实验结果的柱状图。
2. 你对上述实验结果的分析。

![总结](./image.png)

1、缓冲区大小非常关键。

2、对其优化价值：对内存受限的系统效果显著

3、系统调用：fadvise在首次读取大文件更有效

4、优化平衡：mycat5在复杂度和性能间取得平衡

In [ ]:
# 这里填入你用于画图的python代码
data = {
    "Implementation": ['mycat2', 'mycat3', 'mycat4', 'mycat5', 'mycat6', 'system cat'],
    "Time (ms)": [289.3, 282.7, 280.7, 171.1, 174.1, 150.0],
    "Speedup": [1.0, 1.02, 1.03, 1.69, 1.66, 1.93]
}

base_time = data["Time (ms)"][0]
data["Speedup"] = [base_time / t for t in data["Time (ms)"]]

print("\ncat实现性能比较")
print("=" * 65)
print(f"{'实现':<12} | {'时间 (ms)':>10} | {'相对性能':>12} | {'提升倍数':>12} | {'性能柱状图'}")
print("-" * 65)

for i in range(len(data["Implementation"])):
    impl = data["Implementation"][i]
    time_val = data["Time (ms)"][i]
    speedup = data["Speedup"][i]
    
    bar_length = int(speedup * 20)
    bar = '█' * bar_length
    
    print(f"{impl:<12} | {time_val:>10.1f} | {speedup:>12.2f}x | {speedup:>12.2f}x | {bar}")

print("=" * 65)
print("注：相对性能和提升倍数均基于mycat2作为基准")
